# Import

In [1]:
import matplotlib.pyplot as plt
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import davies_bouldin_score, calinski_harabasz_score, silhouette_score
import optuna
from joblib import Parallel, delayed
from utils.Loader import NEUDataset
from utils.Perspectiver import Perspectiver
from source.Prototype1 import Prototype1
from source.Classifier import MetalClassifier

from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam

import multiprocessing
import torch
from mpl_toolkits.mplot3d import Axes3D
from skimage.restoration import denoise_wavelet
import random
import math
from PIL import Image
from collections import deque
import numpy as np
from scipy.ndimage import maximum_filter, minimum_filter, label, generate_binary_structure
from concurrent.futures import ProcessPoolExecutor, as_completed
from scipy.ndimage import label as ndi_label, binary_dilation

/home/liingfeng/Desktop/pytorch/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Init Variables

In [3]:
model = Prototype1(num_attention_heads=16).to("cuda")
if torch.cuda.is_available():
    print("Using cuda cores")
    model.cuda()

dataset = NEUDataset(set="train", seed=555, scale=0.5, best_param=True, output_path="outputs_k10")

Using cuda cores
Dataset: train created!


# Hyper parameters

In [4]:
LEARNING_RATE = 0.0005
EPOCH = 1000
BATCH_SIZE = len(dataset)//2
THREADS = 8

# Training process
Best loss: '2532'

In [5]:
criterion = nn.MSELoss(reduction="sum")
optimizer = Adam(model.parameters(), lr=LEARNING_RATE)
dataloader = DataLoader(dataset=dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=THREADS)

In [ ]:
def train(model):
    loss_record = []
    for epoch in range(EPOCH):
        model.train()
        total_loss = 0
        num_batches = 0

        # Desempaquetar ignorando 'labels'
        for images, _, best_parameters in dataloader:
            images = images.to('cuda')
            best_parameters = best_parameters.to('cuda')

            # Forward: el modelo predice sp y sr
            pred_reg = model(images)  # salida de regresión

            # Calcular la pérdida de regresión
            loss = criterion(pred_reg, best_parameters)
            total_loss += loss.item()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            num_batches += 1

        avg_loss = total_loss / num_batches
        loss_record.append(avg_loss)
        print(f"Epoch [{epoch + 1}/{EPOCH}], Loss: {avg_loss:.4f}")
    return loss_record, model

loss, model = train(model)

In [ ]:
total_parametros = sum(p.numel() for p in model.parameters())
print(f"Model parameters: {total_parametros}")

In [ ]:
plt.plot(loss)           
plt.title("Line Chart")  
plt.xlabel("epoch")     
plt.ylabel("loss")      
plt.show()               

In [ ]:
#Guardar solo la red neuronal en un archivo .pth
torch.save(model.state_dict(), "h2.pth")
print("Modelo guardado exitosamente en h2.pth")

In [ ]:
loaded_model = Prototype1(num_attention_heads=16)
loaded_model.load_state_dict(torch.load("h2.pth", map_location=torch.device('cpu')))
loaded_model.to("cuda")
loaded_model.eval()

# Testing

In [5]:
def plot_barchartImage(image):
    x = np.arange(image.shape[0])
    y = np.arange(image.shape[1])
    x, y = np.meshgrid(x, y)

    # Flatten arrays for plotting
    x = x.flatten()
    y = y.flatten()
    z = np.zeros_like(x)
    dx = dy = np.ones_like(x)
    dz = image.flatten()

    # Plot the 3D bar chart
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')
    ax.bar3d(x, y, z, dx, dy, dz, shade=True)

    # Add labels and title
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Value')
    ax.set_title('3D Bar Chart of (200, 200) Array')

    plt.show()

def use_model_to_cluster(model: nn.Module, image: torch.tensor):
    best_parameters = model(image.to("cuda"))
    print(best_parameters)
    sp = best_parameters[0][0]
    if sp <= 1: sp = 1
    sr = best_parameters[0][1]
    if sr <= 1: sr = 1
    return sp, sr

def plot_how_the_model_is_watching_the_picture(model: nn.Module, dataset: NEUDataset):
    image, label = dataset.__getitem__(index= random.randint(0, len(dataset)))
    print(label)
    original_image = Perspectiver.grayscale_to_rgb(Perspectiver.normalize_to_uint8(image.detach().cpu().numpy()[0]))
    sp , sr = use_model_to_cluster(loaded_model, image)
    clustered_image = Perspectiver.meanShift(original_image, float(sp), float(sr))
    Perspectiver.plotComparison(imageBefore=original_image, imageAfter=clustered_image)

In [6]:
test_set = NEUDataset(set="test", seed=55, scale=0.5, best_param=False)
#plot_how_the_model_is_watching_the_picture(loaded_model, test_set)

Dataset: test created!


# Using knowledge

In [7]:
LEARNING_RATE = 0.00005
EPOCH = 1000
BATCH_SIZE = len(dataset)//4
THREADS = 14
dataset = NEUDataset(set="train", seed=555, scale=0.5)
print(len(dataset))
dataloader = DataLoader(dataset=dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=THREADS)

Dataset: train created!
1656


In [8]:
def train(classifier, dataloader, criterion, optimizer):
    loss_record = []
    classifier.train()
    for epoch in range(EPOCH):
        total_loss = 0
        num_batches = 0

        # Desempaquetar ignorando 'labels'
        for images, label in dataloader:
            images = images.to('cuda')
            label = label[:, 0:6].float().to(device)

            # Forward: el modelo predice sp y sr
            pred_reg = classifier(images)  # salida de regresión

            #print(pred_reg.shape)
            #print(label.shape)

            loss = criterion(pred_reg, label)
            total_loss += loss.item()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            num_batches += 1

            #Opcional: imprimir para depurar
            #print("Logits:", pred_reg)
            #print("Labels:", label)
            #print("Batch Loss:", loss.item())

        avg_loss = total_loss / num_batches
        loss_record.append(avg_loss)
        print(f"Epoch [{epoch + 1}/{EPOCH}], Loss: {avg_loss:.4f}")
    return loss_record, classifier

Best:  num_feature_extractors=2, features_per_extractor=6 ---> loss 24.012
Best:  num_feature_extractors=2, features_per_extractor=8 ---> loss 3.0934

In [42]:
classifier =  MetalClassifier(output_len=6, num_feature_extractors=2, features_per_extractor=8).to("cuda")
if torch.cuda.is_available():
    print("Using cuda cores")
    classifier.cuda()

total_parametros = sum(p.numel() for p in classifier.parameters())
print(f"Model parameters: {total_parametros}")

Using cuda cores
Model parameters: 1175448


In [43]:
loss, classifier = train(classifier=classifier, dataloader=dataloader, criterion=nn.CrossEntropyLoss(reduction="sum"), optimizer=torch.optim.Adam(classifier.parameters(), lr=LEARNING_RATE))

Epoch [1/1000], Loss: 495.1255
Epoch [2/1000], Loss: 495.1058
Epoch [3/1000], Loss: 495.0918
Epoch [4/1000], Loss: 495.0735
Epoch [5/1000], Loss: 495.0539
Epoch [6/1000], Loss: 495.0431
Epoch [7/1000], Loss: 495.0275
Epoch [8/1000], Loss: 495.0099
Epoch [9/1000], Loss: 494.9946
Epoch [10/1000], Loss: 494.9822
Epoch [11/1000], Loss: 494.9691
Epoch [12/1000], Loss: 494.9567
Epoch [13/1000], Loss: 494.9424
Epoch [14/1000], Loss: 494.9321
Epoch [15/1000], Loss: 494.9171
Epoch [16/1000], Loss: 494.9064
Epoch [17/1000], Loss: 494.8933


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x73fb4d8f8220>
Traceback (most recent call last):
  File "/home/liingfeng/Desktop/pytorch/lib/python3.12/site-packages/torch/utils/data/dataloader.py", line 1604, in __del__
    self._shutdown_workers()
  File "/home/liingfeng/Desktop/pytorch/lib/python3.12/site-packages/torch/utils/data/dataloader.py", line 1562, in _shutdown_workers
    if self._persistent_workers or self._workers_status[worker_id]:
                                   ^^^^^^^^^^^^^^^^^^^^
AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute '_workers_status'


Epoch [18/1000], Loss: 494.8817
Epoch [19/1000], Loss: 494.8721
Epoch [20/1000], Loss: 494.8590
Epoch [21/1000], Loss: 494.8500
Epoch [22/1000], Loss: 494.8397
Epoch [23/1000], Loss: 494.8307
Epoch [24/1000], Loss: 494.8201
Epoch [25/1000], Loss: 494.8102
Epoch [26/1000], Loss: 494.8001
Epoch [27/1000], Loss: 494.7910
Epoch [28/1000], Loss: 494.7809
Epoch [29/1000], Loss: 494.7721
Epoch [30/1000], Loss: 494.7630
Epoch [31/1000], Loss: 494.7499
Epoch [32/1000], Loss: 494.7354
Epoch [33/1000], Loss: 494.7119
Epoch [34/1000], Loss: 494.6834
Epoch [35/1000], Loss: 494.6327
Epoch [36/1000], Loss: 494.5558
Epoch [37/1000], Loss: 494.4266
Epoch [38/1000], Loss: 494.2317
Epoch [39/1000], Loss: 493.9205
Epoch [40/1000], Loss: 493.3940
Epoch [41/1000], Loss: 492.4295
Epoch [42/1000], Loss: 490.5197
Epoch [43/1000], Loss: 486.7846
Epoch [44/1000], Loss: 478.2816
Epoch [45/1000], Loss: 463.9382
Epoch [46/1000], Loss: 443.9004
Epoch [47/1000], Loss: 427.0653
Epoch [48/1000], Loss: 420.2417
Epoch [4

In [13]:
plt.plot(loss)           
plt.title("Line Chart")  
plt.xlabel("epoch")     
plt.ylabel("loss")      
plt.show()       

NameError: name 'loss' is not defined

# Testing classifier

In [39]:
def prediction(image):
    # Envía la imagen a GPU, obtiene logits y selecciona la clase predicha.
    output = classifier(image.to("cuda"))
    pred_class = torch.argmax(output, dim=1)
    return pred_class

# Seleccionar una imagen aleatoria del conjunto de prueba
idx = random.randint(0, len(test_set) - 1)
image, label = test_set[idx]

# Evaluar sin calcular gradientes
with torch.no_grad():
    pred = prediction(image)

# Convertir etiqueta one-hot a índice si es necesario
if label.numel() > 1:
    true_class = torch.argmax(label)
else:
    true_class = label

print("Predicción:", pred.item())
print("Etiqueta real:", true_class.item())

Predicción: 0
Etiqueta real: 0


In [32]:
from sklearn.metrics import precision_score

# Listas para almacenar etiquetas verdaderas y predichas
true_labels = []
pred_labels = []

# Colocar el modelo en modo evaluación
classifier.eval()

# Desactivar cálculo de gradientes durante la evaluación
with torch.no_grad():
    for image, label in test_set:
        # Obtener la predicción (se asume que 'prediction' ya mueve la imagen a GPU)
        pred = prediction(image)
        
        # Convertir la etiqueta one-hot a índice, si es necesario
        if isinstance(label, torch.Tensor):
            label_idx = torch.argmax(label).item() if label.numel() > 1 else label.item()
        else:
            label_idx = label
        
        true_labels.append(label_idx)
        pred_labels.append(pred.item())

# Calcular la precisión (macro average)
precision = precision_score(true_labels, pred_labels, average='macro')
print("Precisión del modelo:", precision)


Precisión del modelo: 0.8818764568764569
